# Homework 1: Feed-forward Neural Networks (100 points)

### Overview

Below you will find a PyTorch implementation of a feed-forward neural network for image recognition. We use the popular MNIST dataset, where the model predicts a single digit (0-9) for a black-and-white photo of a handwritten digit. This is a _classification_ task.

### NN Architecture

Each image has size 28x28 grayscale pixel values between 0 and 255. In preprocessing, we flatten each image to a single vector of length $28^2 = 784$, which serves as the entire input for the model.

For each image, we aim to predict one of ten classes (0-9). We could use an output layer $y$ of size 1 (a single neuron) -- for example, using a naive mapping like prediction $p = \mathrm{int}(10y)$. But this presupposes that a handwritten 0 is similar to a handwritten 1 and very different than a handwritten 9, which isn't the case. So instead we use an output layer $y$ of size 10, where the prediction $p = argmax(y)$, so each output neuron controls the likelihood for a particular class.

We use a simple two-layer neural network. To begin, we will have an input size of 784, a hidden layer of size 5, and an output layer of size 10.

### Your Task

At the bottom of this notebook file, there are a series of questions testing your understanding of this neural network architecture. Some questions include instructions where you will need to modify hyperparameters (notated in the code below) and re-run the model to investigate the changed results. __There is no need to read through the following code in depth to answer the questions, but it may be useful as a reference.__

Below each question is a cell with the text “Type Markdown and LaTex.” Double-click the cell and type your response to the question. Save your responses by clicking on the floppy disk icon or choosing File - Save and Checkpoint.

After responding to the questions, download your notebook as a `.html` file by choosing File - Download as - html (.html). You will be submitting this `.html` file to your instructor for grading.

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
torch.manual_seed(0)

In [2]:
root_dir = 'assets_week1'
trainDataset = datasets.MNIST(root=root_dir, train=True, transform=transforms.ToTensor(), download=True)
testDataset = datasets.MNIST(root=root_dir, train=False, transform=transforms.ToTensor())

In [3]:
class NNModel(nn.Module):
    def __init__(self, inputSize, outputSize, hiddenSize, activate):
        super().__init__()
        
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.Tanh() if activate == "Tanh" else nn.ReLU()
        self.layer1 = nn.Linear(inputSize, hiddenSize)
        self.layer2 = nn.Linear(hiddenSize, outputSize)
        
    def forward(self, X):
        hidden = self.activate(self.layer1(X))
        return self.layer2(hidden)

In [4]:
# The dimensionality of the input
inputSize = 784
# Number of neurons in the first layer
# hiddenSize = 5
hiddenSize=300
# Number of neurons in the second layer
outputSize = 10
# Activation function (default: ReLU, options: Sigmoid, Tanh, ReLU)
activation = "ReLU"
# Learning rate
learningRate = 0.001
# learningRate=1
# Number of training epochs
numEpochs = 5
# Number of training examples per batch
batchSize = 200

In [5]:
trainLoader = torch.utils.data.DataLoader(dataset=trainDataset, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=testDataset, batch_size=batchSize, shuffle=False)

net = NNModel(inputSize, outputSize, hiddenSize, activation)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learningRate)

print('>>> Beginning training!')
for epoch in range(numEpochs):
    for i, (images, labels) in enumerate(trainLoader):
        images = images.view(-1, 28*28)
        
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = net(images)
        
        # Backpropagation
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient descent
        optimizer.step()
        
        # Logging
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'.format(epoch+1, numEpochs, i+1, len(trainDataset)//batchSize, loss))

print()
print('>>> Beginning validation!')
correct, total = 0, 0
for i, (images, labels) in enumerate(testLoader):
    images = images.view(-1, 28*28)
    
    outputs = net(images)
    _, prediction = torch.max(outputs, axis=1)
    correct += torch.sum(prediction == labels)
    total += labels.size(0)
print('Validation accuracy: {}%'.format(correct.item()/total*100))

>>> Beginning training!
Epoch [1/5], Step [100/300], Loss: 0.3058919608592987
Epoch [1/5], Step [200/300], Loss: 0.4036794602870941
Epoch [1/5], Step [300/300], Loss: 0.21386142075061798
Epoch [2/5], Step [100/300], Loss: 0.19276389479637146
Epoch [2/5], Step [200/300], Loss: 0.1598106026649475
Epoch [2/5], Step [300/300], Loss: 0.12064734101295471
Epoch [3/5], Step [100/300], Loss: 0.12372788786888123
Epoch [3/5], Step [200/300], Loss: 0.06980592012405396
Epoch [3/5], Step [300/300], Loss: 0.09972573071718216
Epoch [4/5], Step [100/300], Loss: 0.08776193857192993
Epoch [4/5], Step [200/300], Loss: 0.11500314623117447
Epoch [4/5], Step [300/300], Loss: 0.10462900996208191
Epoch [5/5], Step [100/300], Loss: 0.07124170660972595
Epoch [5/5], Step [200/300], Loss: 0.07135775685310364
Epoch [5/5], Step [300/300], Loss: 0.05400325357913971

>>> Beginning validation!
Validation accuracy: 97.50999999999999%


## Homework Questions

Your goal is to improve the model's accuracy by tuning hyperparameters. If a question asks you to modify a hyperparameter and you obtain improved results, retain that hyperparameter change for subsequent questions. Otherwise, revert back to the original hyperparameter value.

**To make sure your code produces consistent results, it is advisable to click "Kernel -> Restart & Run All" every time you want to run your code.**

### Question 1: Loss Minimization & Gradient Descent (5 points)

Given a neural network with model parameters $\theta$, loss function $E$, and learning rate $\alpha$, what is the correct method to perform gradient descent?

a) $\theta_i += \alpha E$

b) $\theta_i -= \alpha E$

c) $\theta_i += \alpha\frac{\partial E}{\partial \theta_i}$

d) $\theta_i -= \alpha\frac{\partial E}{\partial \theta_i}$

**Answer:**

d) $\theta_i -= \alpha\frac{\partial E}{\partial \theta_i}$

### Question 2: Class Imbalance (10 points)

Imagine you are an engineer tasked with helping a company to identify faulty parts early using an machine learning-based image recognition system. What evaluation metric would you use? More specifically, explain why a raw percent accuracy score would be a poor choice of evaluation metric for this problem space.

**Answer:**

For this particular task, I would choose F-score (which combines Precision and Recall in its formula) as an evaluation metric since it would better explain the performance of the model in this case.

A raw percent accuracy score would be a poor choice of evaluation metric for this problem space since it is an imbalanced classification problem, with a clear majority class (non-faulty parts) and minority class (faulty parts). When a dummy model which would only predict the majority class is used as a baseline, the accuracy score in this case would always be very high since it is a measure of the correct predictions divided by the total predictions made by the model (none of which would be for the minority class).

### Question 3a:  Size of a Hidden Layer (10 points)

Explain how the hidden layer size influences the architecture of a feed-forward neural network. In doing so, note what can happen if the hidden size is too large and what can happen if the hidden size is too small.

**Answer:**

The size of the hidden layer influences the fitting of functions of various levels of complexity and the number of parameters that the model has to learn.

A hidden layer size that is too large enables the fitting of more complex functions. However, this comes at the cost of higher computational resources required to learn the large number of parameters. Also, this configuration is prone to overfitting as it would fit the training data perfectly even if it's noisy, lowering its generalization capability for unseen data. On the other hand, a smaller hidden layer size would lead to better generalization capability of the model to unseen data. However, if the hidden layer size is too small, it might not lead to a good fit for the data as the curve becomes extremely simplistic.

### Question 3b: Size of a Hidden Layer  (10 points)

Increase the hidden size from 5 to 300 and re-run your trial. How does the accuracy change?

_a) It increases, since the model learns more quickly_

_b) It increases, since the model has more memory and can learn more complex features_

_c) It decreases, since the model has to learn more parameters and it doesn't have enough time_

_d) It decreases, since the model has less memory_

**Answer:**

b) It increases, since the model has more memory and can learn more complex features

### Question 4a: Learning Rate  (10 points)

Explain the purpose of a learning rate. In doing so, note what can happen if the learning rate is too large and what can happen if the learning rate is too small.

**Answer:**

Learning rate is a hyperparameter that influences how much the weights of the neural network can be adjusted with respect to the loss gradient. 

If the learning rate is too large, the model requires fewer training epochs but it could cause the model to overshoot and diverge from the optimal solution. If the learning rate is too small, the model requires more training epochs and may take a very long time to converge, and may still converge to a bad local optima.

### Question 4b: Learning Rate  (10 points)

Increase the learning rate from 0.001 to 1. How does the accuracy change?

a) It increases, since the model learns more quickly

b) It increases, since the model is better able to converge

c) It decreases, since the model learns too slowly

d) It decreases, since the model is not able to converge

**Answer:**

d) It decreases, since the model is not able to converge

### Question 5a: Activation Functions (10 points)

Explain the main purpose of an activation function in neural networks. Also, explain the main benefit of the Tanh activation function over the Sigmoid activation function, and the main benefit of the ReLU activation function over the Sigmoid activation function.

**Answer:**

The activation function in neural networks decides the activation state of a neuron by implementing a non-linear transformation of the input to enable it to learn and perform more complex tasks. It stipulates the manner of transformation of the weighted sum of the input into an output from a node. The choice of non-linear activation function has a large impact on the performance of a neural network.

The main benefit of the tanh activation function over the sigmoid activation function is that the cost function is minimized faster on using the tanh activation function since the derivatives of tanh are larger than those of the sigmoid activation function. The outputs of the former are between -1 and 1 while those of the latter are between 0 and 1, which means that even negative values are considered in the case of tanh. 

The main benefit of the ReLU activation function over the sigmoid activation function is that the derivative of the ReLU activation function is computed faster than that of the sigmoid activation function since the ReLU activation function just needs to pick a maximum value of 0 or x (it changes all negative values to 0) and does not need to perform an expensive exponential operation as in the case of sigmoid activation function.

### Question 5b: Activation Functions (5 points)

Change the activation function in the hyperparameter list above to determine which activation function is most effective at this task.

a) ReLU

b) Sigmoid

c) Tanh

**Answer:**

a) ReLU

### Question 6: Overfitting  (10 points)

Define overfitting and explain how it can damage model training and results.

Overfitting is the phenomenon wherein the model memorizes the training data and fits to it perfectly, often at the consequence of a suboptimal performance with unseen data. A very complex model which learns the training data perfectly (including noise in the data, if any), as opposed to learning the general curve of the underlying pattern of the data is unable to generalize well to data it hasn't been exposed to before, leading to a poor score with the test dataset while achieving a deceptively high score with the training dataset. An overfit model, if undetected during development due to misleadingly good scores, would likely lead to dire consequences when implemented in production as it would not be able to provide a good fit for the new data that it would be exposed to and would likely lose its value as a predictive tool in a practical real-world setting.

### Question 7: Early Stopping  (10 points)

Outline a procedure for early stopping to prevent overfitting. Clearly describe how you’d use the training, validation, and test sets accuracy to decide when to stop.

**Answer:**

Early Stopping is a method to stop training the model at a certain instant in order to prevent it from overfitting to the training data. The following are the steps I would use to implement Early Stopping:

* Divide the data into training, validation, and test sets in the ratio of 6:2:1. The training set would be used to train the data, the validation set would be used to evaluate the loss of the model on every epoch, and the test set would be a hold-out set (not exposed to model during training) for the final model evaluation.


* Define the model architecture by specifying the type of model, the number of layers, the number of nodes in each of the layers, the activation functions, the loss function, the optimizer, and the evaluation metric (here, accuracy) depending on the problem to be solved.


* Train the model on a high number of epochs (say, 5000) and plot the loss of the model on both, the training set and the validation set for each epoch (loss can be found out by using the history attribute if Keras is being used). Typically, the loss in both cases would start off high and drop down as the number of training epochs increase. If the loss plot line of the training set drops very much lower than that of the validation set (i.e. the performance of the model is better on the training set than on the validation set in terms of accuracy), the model is likely overfitting.


* Observe the loss plot line ōf the validation set. In an overfit model, the loss plot line would start off high, drop down until a certain number of epochs is reached, and then rise up again (while that of the training set continues to drop). The point at which the rise in loss value for the validation set happens indicates the early stopping point (the point of highest accuracy and lowest loss on the validation set), which would be a good point to stop the training of the model and prevent it from overfitting.


* Retrain the model by stopping the training of the model at the early stopping point determined from step 4. This can either be done by updating the number of training epochs to reflect this early stopping point, or by using the EarlyStoppingPoint class object within the model architecture if Keras is being used.


* Finally, evaluate the performance of this trained model with the hold-out test set using a suitable evaluation metric (here, accuracy).

### Question 8: Regularization  (10 points)

Briefly explain a few common methods of regularization to prevent overfitting.

**Answer:**

The following are some of the common methods of regularization to prevent overfitting:

* Dropout: During each iteration, this technique randomly drops neurons from the neural network, which effectively makes it equivalent to training several different neural networks, whose net effect would be to reduce overfitting. This makes the model more robust and ensures that it doesn't rely solely on one configuration of neurons to make a prediction. 


* Early Stopping: This is a technique of stopping the training of the model at a certain point, beyond which the model would overfit. The point of early stopping is the point just before which the validation loss has started to increase while the training loss is still decreasing. This would ensure that the scores on the validation data are as good as they can be while still preventing the model from overfitting.


* Data Augmentation: This technique entails augmenting the training data with artificially generated data which involves transforming the input data (e.g. images) such that the original class label of this data is preserved. The number of images in the training data can be increased by multiple techniques such as rotation, cropping, blurring, flipping, etc. This is done so that the model can become more robust in identifying the diverse features and nuances of the training data, which would prevent it from memorizing the training data and overfitting.


* L1 and L2 Regularization: With these techniques, a penalty term is added to the loss function since they work on the basis that smaller weights give rise to simpler models, which helps in preventing overfitting. L1 penalizes the absolute sum of weights while L2 penalizes the sum of squared values of weights. While both techniques are useful in preventing overfitting depending on the case, L1 is generally used in cases where the data can be interpreted with a simple model, while L2 is able to learn complex inherent patterns in the data.